In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import re

In [4]:
data = pd.read_csv('tripadvisor_hotel_reviews.csv')

In [5]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [9]:
data.describe(include=['O'])

,Review
count,20491
unique,20491
top,target absolutely customer centric experience ...
freq,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [14]:
X = data['Review']
y = data['Rating']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

### Topic Modeling

Usually, In the hotel industry. The Enterprenue recived the useful information form data in text uniform such as The comment in social network or email from the customers. 
The Enterprenue may use the useful information and group the useful information for analysis or make the the better goods and 
service

In this section. topic modeling usually use for anaylyze what the topic of the review is?

In [67]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
def text_process(text):
    text = re.sub('[^A-za-z]'," ",text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stopwords.words('english')]
#     text = [SnowballStemmer('english').stem(word) for word in text]
    text = " ".join(text)
    return text

In [68]:
X_process = X.apply(text_process)

In [205]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

## Preprocess Text to count vector
cv = CountVectorizer(max_df=0.9,min_df=3)
cv.fit(X_process)
X_vec = cv.transform(X_process.iloc[:]).toarray()

# tf = TfidfVectorizer(max_df=0.9,min_df=2)
# X_tf = tf.fit_transform(X_process).toarray()

In [206]:
## Latent Dirichlet Allocation
lda = LatentDirichletAllocation(n_components=7)
lda.fit(X_vec)

LatentDirichletAllocation(n_components=7)

#### Allocate the topic to vector


In [207]:
word_topic = {}
for i,w in enumerate(lda.components_):
    word_topic[f'topic_{i}'] = [cv.get_feature_names()[j] for j in lda.components_[i].argsort()[-10:]]

In [208]:
word_topic

{'topic_0': ['rooms',
  'clean',
  'staff',
  'stay',
  'good',
  'nice',
  'location',
  'great',
  'room',
  'hotel'],
 'topic_1': ['place',
  'restaurant',
  'night',
  'food',
  'water',
  'got',
  'service',
  'like',
  'day',
  'room'],
 'topic_2': ['rooms',
  'best',
  'wonderful',
  'stayed',
  'service',
  'stay',
  'room',
  'staff',
  'great',
  'hotel'],
 'topic_3': ['check',
  'told',
  'rooms',
  'staff',
  'service',
  'night',
  'desk',
  'stay',
  'hotel',
  'room'],
 'topic_4': ['room',
  'day',
  'people',
  'time',
  'pool',
  'great',
  'good',
  'food',
  'resort',
  'beach'],
 'topic_5': ['stay',
  'clean',
  'rooms',
  'great',
  'staff',
  'location',
  'breakfast',
  'good',
  'room',
  'hotel'],
 'topic_6': ['area',
  'rooms',
  'juan',
  'san',
  'nice',
  'room',
  'great',
  'pool',
  'beach',
  'hotel']}

In [209]:
topic_model = lda.transform(X_vec)

In [215]:
topic_model.argmax(axis=1)

array([0, 3, 0, ..., 0, 3, 5], dtype=int64)